In [266]:
from uuid import uuid1
from lxml import html
import pandas as pd
import json
import matplotlib.pyplot as plt
import spacy
from spacy import displacy
import numpy as np
import warnings
from pprint import pprint
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline
from reddit import load
from bs4 import BeautifulSoup

In [267]:


# add additional subreddits below or comment them out with #
subreddits = """\
https://www.reddit.com/r/AirBnB/
#https://www.reddit.com/r/AmazonFlexDrivers/
#https://www.reddit.com/r/Etsy/
#https://www.reddit.com/r/InstacartShoppers/
#https://www.reddit.com/r/TaskRabbit
#https://www.reddit.com/r/beermoney/
#https://www.reddit.com/r/couriersofreddit
#https://www.reddit.com/r/doordash/
#https://www.reddit.com/r/freelance/
#https://www.reddit.com/r/lyftdrivers/
#https://www.reddit.com/r/turo/
#https://www.reddit.com/r/uberdrivers/
#https://www.reddit.com/r/ridesharedrivers/
"""

data = load(subreddits.split())

loaded 1000 reddit submissions


In [268]:
nlp = spacy.load('en_core_web_md') 

In [271]:
pprint(data[0]['url'])

'https://www.reddit.com/r/AirBnB/comments/ccg9iq/hey_guest_people_i_live_on_the_property_i/'


In [272]:
submissions = {}
for pos, i in enumerate(data):
    title_id = uuid1()
    title = i['title']
    text = i['text']
    upvote_ratio = i['upvote_ratio']
    score = i['score']
    created_utc = i['created_utc']
    url = i['url']
    permalink = i['permalink']
    is_self = i['is_self']
    num_comments = i['num_comments']
    submission = {"title" : title, "text" : text, "upvote_ratio": upvote_ratio
                  , "score": score, "created_utc": created_utc, "permalink": permalink
                  , "is_self": is_self, "num_comments": num_comments}
    submissions[title_id] = submission

In [273]:
data_df = pd.DataFrame.from_dict(submissions, orient = "index")
data_df.to_csv("titles_df")

In [95]:
import pandas
titles_df = pd.read_csv("titles_df")
titles_df['pos'] = list(titles_df.index)
titles_df = titles_df.rename(columns={"Unnamed: 0": "title_id"})
titles_df = titles_df.set_index("title_id")
titles_df.head()

,title,text,upvote_ratio,score,created_utc,permalink,is_self,num_comments,pos
title_id,,,,,,,,,
adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,"Hey guest people, I live on the property, I in...","...I am a phone call away, if you need somethi...",0.99,255,1.562964e+09,/r/AirBnB/comments/ccg9iq/hey_guest_people_i_l...,True,80,0
adac8824-54bd-11ea-8a4b-9cb6d0bc58b0,Good Bye Airbnb! Its been a great 8 years!,"I have been a host on Airbnb for 8 years now, ...",0.98,197,1.562134e+09,/r/AirBnB/comments/c8khhc/good_bye_airbnb_its_...,True,80,1
adac913e-54bd-11ea-8a4b-9cb6d0bc58b0,My guest story for the day...,"So today, I get a booking at 3:05 PM for my pr...",0.98,193,1.573008e+09,/r/AirBnB/comments/ds9z7l/my_guest_story_for_t...,True,100,2
adac9396-54bd-11ea-8a4b-9cb6d0bc58b0,Guest lied and threw a massive after-prom part...,TL:DR: My wife and I are superhosts that were ...,0.98,190,1.526414e+09,/r/AirBnB/comments/8jonnz/guest_lied_and_threw...,True,73,3
adac95d0-54bd-11ea-8a4b-9cb6d0bc58b0,I've been living in AirBnB's for over a year. ...,Hello everyone. Over the past year and a half...,0.98,188,1.546046e+09,/r/AirBnB/comments/aaggep/ive_been_living_in_a...,True,44,4


In [274]:
all_comments = {}
for pos, title_id in zip(titles_df.pos, titles_df.index):
    comments = data[pos]['comments']
    for j in comments:
        comment_id = uuid1()
        body = j['body']
        created_utc = j['created_utc']
        score = j['score']
        comment_row = {"title_id": title_id, "body": body, "created_utc": created_utc, "score": score}
        all_comments[comment_id] = comment_row
        

In [276]:
comment_df = pd.DataFrame.from_dict(all_comments, orient = "index")
comment_df

,title_id,body,created_utc,score
fa71e2f6-54d8-11ea-8a4b-9cb6d0bc58b0,adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,You said lobster crackers but I thought of oys...,1.562966e+09,72
fa71ed6e-54d8-11ea-8a4b-9cb6d0bc58b0,adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,Yeah I can relate. Though sometimes it's never...,1.562974e+09,63
fa71ef44-54d8-11ea-8a4b-9cb6d0bc58b0,adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,I vibe this 100%. In the review: “A lightbulb ...,1.562965e+09,87
fa71f0f2-54d8-11ea-8a4b-9cb6d0bc58b0,adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,"""Missing Amenities: No Towels"" \n\n Or from a...",1.562969e+09,69
fa71f282-54d8-11ea-8a4b-9cb6d0bc58b0,adac7eba-54bd-11ea-8a4b-9cb6d0bc58b0,"""The hall light was on so we couldn't sleep."" ...",1.562969e+09,12
...,...,...,...,...
facdc602-54d8-11ea-8a4b-9cb6d0bc58b0,adb2f1e6-54bd-11ea-8a4b-9cb6d0bc58b0,"Final update, he offered £350 which was about ...",1.573555e+09,3
facdc760-54d8-11ea-8a4b-9cb6d0bc58b0,adb2f1e6-54bd-11ea-8a4b-9cb6d0bc58b0,Following,1.573336e+09,2
facdc8fa-54d8-11ea-8a4b-9cb6d0bc58b0,adb2f1e6-54bd-11ea-8a4b-9cb6d0bc58b0,Good luck and keep us posted!\n\nFor clarifica...,1.573337e+09,1
facdca58-54d8-11ea-8a4b-9cb6d0bc58b0,adb2f1e6-54bd-11ea-8a4b-9cb6d0bc58b0,"So Airbnb emailed me this morning:\n\n*Hi Dan,...",1.573396e+09,1


In [216]:
comment_df.iloc[1, :].body

"Yeah I can relate. Though sometimes it's never enough. We once had guests ask if we had bikes during the booking, and I thought 'Yeah bikes would be handy for guests'. So I went and bought two used bikes, serviced them, cleaned them up. Guests arrived, I set them up and got them fitted.\n\nNext day they complained about the bikes. They wanted to cycle 85km each way to another city over. I got them runabout bikes to go get a coffee or cruise the suburb/city/park.... They were actually pretty hardcore cyclists, and they expected me to have them some sweet carbon roadbikes with top end gear? Like what? Im not running the fucking Lance Armstrong Experience. I'm $50 a night...."

In [291]:
comments_gpes = {}
for i, comment_id in zip(comment_df.body, comment_df.index):
    doc = nlp(u"{}".format(i))
    entities = [(j, j.label_, j.label) for j in doc.ents]
    comment_gpe = []
    for ent in entities:
        if ent[1] == 'GPE':
            comment_gpe.append(ent)
    #comments_gpes.append(comment_gpe)
    comments_gpes[comment_id] = comment_gpe

In [296]:
comments_gpe_filter = {}
for key, value in comments_gpes.items():
    if len(value) == 0:
        continue
    comments_gpe_filter[key] = value

In [307]:
test_id = comment_df.loc[UUID('fa745a40-54d8-11ea-8a4b-9cb6d0bc58b0'), 'body']
test_id
#titles_df.loc[test_id, :]

'Not exactly, but lowering the price for cleaning is appealing. We usually attract 2-4 people for our 3 bedrooms/3 bath and they stay for a week or more. If we only have 2 nights available, being able to reduce the cleaning fee would be helpful to get them booked. In particular because In those situations our cleaning fee seems out of whack, because it is as much as the discounted rate and thus discourages shorter bookings. I’d love to strike a balance. Being able to change the cleaning fee and the number of people we’d allow would offer a lot more flexibility for booking.'

In [313]:
unique_gpes = []
for value in comments_gpe_filter.values():
    for gpe in value:
        if str(gpe[0]).upper() not in unique_gpes:
            unique_gpes.append(str(gpe[0]).upper())

In [319]:
spelunker = "https://spelunker.whosonfirst.org/search/?q={}"

In [351]:
wof_ids = []
for i in unique_gpes:
    try:
        r = requests.get(spelunker.format(i))
        soup = BeautifulSoup(r.text)
        list_el = soup.findAll("li", {"class": "loc"})
        wof_id = str(list_el).split(" ")[2].split("\"")[1]
        wof_ids.append(wof_id)
    except:
        wof_ids.append(None)
        print(i)

In [354]:
for gpe, wof_id in zip(unique_gpes, wof_ids):
    print(gpe, wof_id)

MAINE 101749017
’D 85633111
SAN FRANCISCO 101997865
INDIANAPOLIS 85942149
LONDON 101750367
LITTLEFINGER None
POLKA 1293682261
HAWAII 85937559
BARCELONA 101748323
JAPAN 85632429
FLORIDA 85932547
WAIKIKI 101937883
HONOLULU 85937449
ITALY 85633253
AUSTRALIA 85632793
SUPERHOST None
UK 85632645
DC 85670165
HOLLYWOOD 101735835
CANADA 85633041
FRANCE 85633147
SWITZERLAND 85633051
SCOTTSDALE 85917475
USA 85977539
KOREA 85632231
LOS ANGELES 85923517
AIRBNB 588730367
AIRBNBS None
MANHATTAN 85977539
LISBON 101752087
HONG KONG 85632483
US 85632469
VEGAS 85974801
TL;DR 840260847
DUBLIN 1125364277
AMERICA 85633009
WARSAW POLAND 101752777
IKEA 101758873
CASPER 101734031
MEXICO 85633293
GUATEMALA 85632385
PARIS 102027181
NASHVILLE 1125278321
PITTSBURGH 101718805
BC 85669993
BEACH 85923517
LGBT 101983371
SPOKANE 101730021
PORTLAND 101715829
ME 85632609
WYOMING 101734121
PENNSYLVANIA 101718083
MILWAUKEE 101732987
GERMANY 85633111
BERLIN 101748799
MONTREAL 890458661
CHICAGO 85940195
ALASKA 85915707
SF 85

In [357]:
import sqlalchemy

In [363]:
engine = sqlalchemy.create_engine('sqlite://mnt/c/Users/nicholas/Desktop/wof/whosonfirst-data-latest.db')
connection = engine.connect()

ArgumentError: Invalid SQLite URL: sqlite://mnt/c/Users/nicholas/Desktop/wof/whosonfirst-data-latest.db
Valid SQLite URL forms are:
 sqlite:///:memory: (or, sqlite://)
 sqlite:///relative/path/to/file.db
 sqlite:////absolute/path/to/file.db

In [202]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nicholas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [205]:
#/usr/lib/jvm/java-11-openjdk-amd64 -- path to java

# setting JAVAHOME environment variable so nltk knows where to find java
import os
java_path = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ['JAVAHOME'] = java_path

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# path to Stanford NER .jar and classification model
st = StanfordNERTagger('/mnt/c/Users/nicholas/Documents/GitHub/hackathon-2020-reddit-nlp/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/mnt/c/Users/nicholas/Documents/GitHub/hackathon-2020-reddit-nlp/stanford-ner-2018-10-16/stanford-ner.jar',
                        encoding='utf-8')

In [233]:
text ="""I was gonna post my awesome experience, but instead I’m gonna hop on here to share instead of creating a whole new post.\n\nThis has been my first experience with Airbnb.  I decided to stay 2 nights in Portland, ME to make use of an extended weekend vacation I took from work. I booked 2 nights in a room in the host’s apartment/condo, tonight is my second and last night here. My host has been nothing but amazing! Extremely friendly and welcoming, full of suggestions on places to check out, and even offered to come hang with me since I’m traveling solo. The place is very clean, I had fresh linens and towels in my room on arrival, and have overall felt like I’m in my own home.  My only complaint is there’s no A/C in my room, but thankfully it’s been cool at night and the box fan that’s in the room has done its job.\n\nI think it’s great to share our good experiences here on Reddit. I followed this sub for a month or two after booking my stay, and some posts made me afraid of what could potentially happen.  For anyone that’s considering using Airbnb, I’m going to take a guess that most people have good experiences. Keep in mind people as a whole like to complain, and this sub is mainly used for troubleshooting.  I easily could’ve spent twice as much for a hotel room during the busy season here in Maine.  I will definitely be using Airbnb again in the future!"""

tokenized_text = word_tokenize("États-Unis")
classified_text = st.tag(tokenized_text)

for i in classified_text:
    if i[1] == 'LOCATION':
        print(i)

In [240]:
doc = nlp("")

entities=[(i, i.label_, i.label) for i in doc.ents]
for i in entities:
    if i[1] == 'GPE':
        print(i)

(Dokdo Island, 'GPE', 384)
